In [1]:
import pandas as pd

In [2]:
sites = pd.read_csv('./Waterbase_v2018_1_WISE3_csv/Waterbase_v2018_1_WISE3_MonitoringSite_DerivedData.csv')

In [3]:
wise3 = pd.read_csv('./Waterbase_v2018_1_WISE3_csv/Waterbase_v2018_1_T_WISE3_MonitoringData.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (5,6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
wise4 = pd.read_csv( './Waterbase_v2018_1_WISE4_csv/Waterbase_v2018_1_WISE4_MonitoringSite_DerivedData.csv')

In [5]:
print(wise3.columns)

water_flow = wise3[['monitoringSiteIdentifier','phenomenonTimePeriod', 'observedProperty', 'resultObservedValue']]\
    .pivot_table(index=['monitoringSiteIdentifier', 'phenomenonTimePeriod'], columns=['observedProperty'], values=['resultObservedValue'])\
    .reset_index()

Index(['monitoringSiteIdentifier', 'monitoringSiteIdentifierScheme',
       'observedProperty', 'phenomenonTimePeriod', 'resultObservedValue',
       'resultObservationStatus', 'Remarks', 'metadata_versionId',
       'metadata_beginLifeSpanVersion', 'metadata_statusCode',
       'metadata_observationStatus', 'metadata_statements', 'UID'],
      dtype='object')


In [6]:
print(wise4.columns)

Index(['monitoringSiteIdentifier', 'monitoringSiteIdentifierScheme',
       'waterBodyIdentifier', 'waterBodyIdentifierScheme',
       'confidentialityStatus', 'lon', 'lat'],
      dtype='object')


In [16]:
merged = water_flow.merge(sites, on='monitoringSiteIdentifier')

merged = merged[[('phenomenonTimePeriod', '' ), ('resultObservedValue', 'GWL'), ('resultObservedValue', 'SF'), 'lat', 'lon']] 
merged.columns = [ 'date', 'ground_water_level', 'stream_flow', 'location.lat', 'location.lon' ]

merged = merged.dropna(subset=['location.lat', 'location.lon'])
merged['location'] = merged['location.lat'].astype('U32') + ',' + merged['location.lon'].astype('U32')
merged.drop(['location.lat', 'location.lon'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [17]:
merged.to_csv('data.csv', index=False)